In [44]:
!pip3 install sklearn
!pip3 install foursquare
!conda install -c conda-forge folium=0.5.0 --yes



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Santiago\anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    conda-4.10.1               |   py38haa244fe_0         3.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1h             |       he774522_0         5.8 MB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    -----------------------------------------------



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda




In [45]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import foursquare
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
import folium

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"6e4b3fa5-1bad-4838-97d7-75bc53a6391d","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1019189119,"wgRevisionId":1019189119,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia

In [4]:
#parse data into dataframe with help from webscraping hint

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
#Delete rows where Borough is 'Not assigned'
df = df[df.Borough != "Not assigned"]

#Combine all neighbourhoods where postcode and Borough are the same
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
#if Neighbourhood is not assigned, replace by borough name
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# define the dataframe columns
column_names = ['Postal Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
df.columns = column_names

df.shape

(103, 3)

In [6]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#reading the geographical data from the provided file

geo_data = pd.read_csv('Geospatial_Coordinates.csv')

In [8]:
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
geo_data.shape

(103, 3)

In [10]:
#merging the geo data with the neighbourhood data into a new "result" dataframe

result = pd.merge(df, geo_data, on="Postal Code")

result.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [11]:
result.shape

(103, 5)

In [12]:
#Foursquare API credentials

CLIENT_ID = 'J35HJF2XRRQS2LYIUQSOC1UZH3Q4Y5TOQ4YSBPOTPMCSFGWY' # your Foursquare ID
CLIENT_SECRET = 'WI3V3OOGST2YQIWLX0DXDJB0ZLFZXAVLLMQDLFIOKSFWRIZA' # your Foursquare Secret
ACCESS_TOKEN = '24DYHTTXPQNTBJ2KU0C0P34PCRO1VSTH1TSWDVH3AOBVYLCS' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

#Instantiating the foursquare client

client = foursquare.Foursquare(access_token=ACCESS_TOKEN)

In [13]:
#test for foursquare API
venues = client.venues.search(params={'ll': '43.806686,-79.194353'})
venues2 = client.venues.search(params={'ll': '43.706748,-79.594054'})

In [14]:
#Function gets list of venue category names for a given venue search result
def unique_venues(x):
    lov = [] #list of string represents the list of venues
    for i in range(1, len(x["venues"])):
        try: 
            lov.append(x["venues"][i]["categories"][0]['name'])
        except IndexError:
            pass
    return lov

In [15]:
#function gets list of all venue categories for all postal code venue searches and appends them in one list:
def get_list_of_venues(x):
    lov = [] #list of string represents the list of venues
    for i in range(1, x.shape[0]):
        lov.append(unique_venues(client.venues.search(params={'ll': str(x.iloc[i]["Latitude"])+","+str(x.iloc[i]["Longitude"])})))
    return lov

In [16]:
#With this call we get a list of all venue categories for all postal code venue searches
total_list = get_list_of_venues(result)

In [17]:
def get_unique_list(x):
    #For each of the postal codes there is a list of venue categories
    #this function takes in a list of all those venue category lists and returns a single
    #concatenated list in order to make duplicate filtering easier.
    lov = []
    for i in range(0 , len(x)):
        for j in range (0, len(x[i])):
            lov.append(x[i][j])
    return lov

In [18]:
#get the unique set of venue categories from all the postal codes in the Toronto Area using the
#data we already have
unique_venues = set(get_unique_list(total_list))
print(set(get_unique_list(total_list)))

{'South Indian Restaurant', 'Filipino Restaurant', 'Train Station', 'Greek Restaurant', 'Thai Restaurant', 'Auto Dealership', 'Post Office', 'Kitchen Supply Store', 'Recycling Facility', 'Business Service', 'Park', 'Restaurant', 'Bank', 'Airport Food Court', 'Flea Market', 'Food & Drink Shop', 'Currency Exchange', 'Smoke Shop', 'Factory', 'Cemetery', 'Cantonese Restaurant', 'Eastern European Restaurant', 'Vegetarian / Vegan Restaurant', 'Tourist Information Center', 'Farmers Market', 'Food Court', 'Farm', 'Bowling Alley', 'Vietnamese Restaurant', 'Italian Restaurant', 'Snack Place', 'Dry Cleaner', 'Liquor Store', 'Frozen Yogurt Shop', 'Lounge', 'Roof Deck', 'Cajun / Creole Restaurant', 'Clothing Store', 'Modern European Restaurant', 'Photography Lab', 'College Classroom', 'College Communications Building', 'General College & University', 'Record Shop', 'Warehouse Store', 'Cupcake Shop', 'Skate Park', 'Financial or Legal Service', 'Dog Run', 'Movie Theater', 'Capitol Building', 'History

In [19]:
print("There are "+str(len(set(get_unique_list(total_list))))+" unique venue categories in toronto")

There are 384 unique venue categories in toronto


In [20]:
#here we get the list of all venues per postal code mapped to a data frame. 
venues_df = DataFrame(total_list)
venues_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Automotive Shop,Church,Gym / Fitness Center,Nail Salon,Bar,Breakfast Spot,Italian Restaurant,BBQ Joint,Burger Joint,Pizza Place,...,Nail Salon,African Restaurant,Breakfast Spot,Italian Restaurant,Hotel,General Entertainment,Church,Mexican Restaurant,None,None
1,Electronics Store,Spa,Automotive Shop,Tech Startup,Fried Chicken Joint,Laundry Service,Restaurant,Donut Shop,Salon / Barbershop,Greek Restaurant,...,Residential Building (Apartment / Condo),Bus Stop,Fast Food Restaurant,Plaza,Residential Building (Apartment / Condo),Mexican Restaurant,Bakery,Salon / Barbershop,None,None
2,Garden,Coffee Shop,Park,Convention Center,Korean BBQ Restaurant,Filipino Restaurant,Chinese Restaurant,Church,Church,Bus Stop,...,Residential Building (Apartment / Condo),Hookah Bar,Indian Restaurant,South Indian Restaurant,South Indian Restaurant,Bus Stop,Bus Line,Restaurant,None,None
3,Building,Miscellaneous Shop,Thai Restaurant,Caribbean Restaurant,Hakka Restaurant,Building,Building,Meeting Room,South Indian Restaurant,Grocery Store,...,Housing Development,Indian Restaurant,Dessert Shop,Skating Rink,Vegetarian / Vegan Restaurant,Burger Joint,Office,None,None,None
4,Bank,Salon / Barbershop,Pizza Place,Playground,Residential Building (Apartment / Condo),Playground,Train Station,School,Chinese Restaurant,Bus Line,...,Automotive Shop,Gas Station,Pool,Automotive Shop,College Academic Building,Convenience Store,Fast Food Restaurant,Fast Food Restaurant,Restaurant,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Convenience Store,Bar,Athletics & Sports,Post Office,Event Space,Middle Eastern Restaurant,Residential Building (Apartment / Condo),Coffee Shop,Park,Office,...,Grocery Store,History Museum,Electronics Store,Hospital,Park,Auto Dealership,Building,Salon / Barbershop,Park,None
98,Cemetery,Pizza Place,Park,Dentist's Office,Chinese Restaurant,Ice Cream Shop,Residential Building (Apartment / Condo),Event Space,Park,Miscellaneous Shop,...,Salon / Barbershop,Gas Station,Auto Dealership,Salon / Barbershop,Park,Veterinarian,Bank,Mobile Phone Shop,Discount Store,None
99,Convenience Store,Mobile Phone Shop,Beer Store,Doctor's Office,Bus Line,Optical Shop,Park,Building,Residential Building (Apartment / Condo),Chinese Restaurant,...,Shopping Mall,Volleyball Court,Park,Professional & Other Places,Building,Laundry Service,Japanese Restaurant,Post Office,None,None
100,Caribbean Restaurant,Dentist's Office,Grocery Store,Farm,Pizza Place,Grocery Store,Clothing Store,Field,Fried Chicken Joint,Farm,...,Movie Theater,Bagel Shop,Movie Theater,Movie Theater,Movie Theater,Farm,Movie Theater,Movie Theater,Movie Theater,None


In [21]:
venues_df.iloc[101]

0                       Drugstore
1                          Office
2                             Bar
3             Rental Car Location
4                         Dog Run
5                         Factory
6                         Factory
7                   Moving Target
8                          Office
9                    Kingdom Hall
10              Afghan Restaurant
11    Professional & Other Places
12                       Building
13                       Building
14                Doctor's Office
15                          Hotel
16                          Hotel
17                            Pub
18         Transportation Service
19                           Café
20                         Church
21                         Office
22                Automotive Shop
23                 Police Station
24                Automotive Shop
25                       Building
26                         Office
27                           None
28                           None
Name: 101, dty

In [22]:
#convert the venue categories into a dictionary to make vectorization simpler for our ML model
venue_categories = dict.fromkeys(unique_venues, 0)
venue_categories

{'South Indian Restaurant': 0,
 'Filipino Restaurant': 0,
 'Train Station': 0,
 'Greek Restaurant': 0,
 'Thai Restaurant': 0,
 'Auto Dealership': 0,
 'Post Office': 0,
 'Kitchen Supply Store': 0,
 'Recycling Facility': 0,
 'Business Service': 0,
 'Park': 0,
 'Restaurant': 0,
 'Bank': 0,
 'Airport Food Court': 0,
 'Flea Market': 0,
 'Food & Drink Shop': 0,
 'Currency Exchange': 0,
 'Smoke Shop': 0,
 'Factory': 0,
 'Cemetery': 0,
 'Cantonese Restaurant': 0,
 'Eastern European Restaurant': 0,
 'Vegetarian / Vegan Restaurant': 0,
 'Tourist Information Center': 0,
 'Farmers Market': 0,
 'Food Court': 0,
 'Farm': 0,
 'Bowling Alley': 0,
 'Vietnamese Restaurant': 0,
 'Italian Restaurant': 0,
 'Snack Place': 0,
 'Dry Cleaner': 0,
 'Liquor Store': 0,
 'Frozen Yogurt Shop': 0,
 'Lounge': 0,
 'Roof Deck': 0,
 'Cajun / Creole Restaurant': 0,
 'Clothing Store': 0,
 'Modern European Restaurant': 0,
 'Photography Lab': 0,
 'College Classroom': 0,
 'College Communications Building': 0,
 'General Colle

In [23]:
def vectorize_row(x):
    # dataframe row -> dictionary
    # function takes a single dataframe row and adds the venues into a dictionary that 
    # keeps the count of each venue type for that given postal code (row)
    vectorized = dict.fromkeys(unique_venues, 0)
    for i in range(0,len(x)):
        if x[i] is None:
            pass
        else:
            vectorized[x[i]] = vectorized[x[i]] + 1
        
    return vectorized

In [24]:
def vectorize_dataframe(x):
    #whole dataframe -> dataframe
    # function takes a whole dataframe and returns another dataframe wich has the count of each
    # venue category per postal code.
    ans = pd.DataFrame([venue_categories])
    for i in range (0,len(x)):
         ans = ans.append(pd.DataFrame([vectorize_row(x.iloc[i])]))
    return ans

In [25]:
ans = pd.DataFrame([vectorize_row(venues_df.iloc[19])])
ans2 = pd.DataFrame([vectorize_row(venues_df.iloc[20])])
ans = ans.append(ans2)
ans

,South Indian Restaurant,Filipino Restaurant,Train Station,Greek Restaurant,Thai Restaurant,Auto Dealership,Post Office,Kitchen Supply Store,Recycling Facility,Business Service,...,Gift Shop,Shopping Plaza,State / Provincial Park,IT Services,Department Store,College Administrative Building,Newsstand,Playground,School,Beer Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
vectorized = vectorize_dataframe(venues_df)
vectorized

,South Indian Restaurant,Filipino Restaurant,Train Station,Greek Restaurant,Thai Restaurant,Auto Dealership,Post Office,Kitchen Supply Store,Recycling Facility,Business Service,...,Gift Shop,Shopping Plaza,State / Provincial Park,IT Services,Department Store,College Administrative Building,Newsstand,Playground,School,Beer Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0,2,1,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,2,0
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
final = vectorized[1:]
final.reset_index(drop=True, inplace=True) #resetting the indexes so we can merge the dataframes
final.shape

,South Indian Restaurant,Filipino Restaurant,Train Station,Greek Restaurant,Thai Restaurant,Auto Dealership,Post Office,Kitchen Supply Store,Recycling Facility,Business Service,...,Gift Shop,Shopping Plaza,State / Provincial Park,IT Services,Department Store,College Administrative Building,Newsstand,Playground,School,Beer Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2,1,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,0,0,2,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,1,0


In [29]:
#merge the vectorized DF with the Location Data in order to make one dataframe to put our ML model on:
ml_df = result.merge(final, left_index=True, right_index=True)
ml_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude,South Indian Restaurant,Filipino Restaurant,Train Station,Greek Restaurant,Thai Restaurant,...,Gift Shop,Shopping Plaza,State / Provincial Park,IT Services,Department Store,College Administrative Building,Newsstand,Playground,School,Beer Store
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,1,0,0,0,...,0,2,0,0,1,0,0,0,2,0
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,0,1,0,0,...,0,0,0,0,0,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
98,M9N,York,Weston,43.706876,-79.518188,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Machine Learning Part and Model

In [80]:
k_means = KMeans(init = "k-means++", n_clusters = 5, n_init = 12)

In [81]:
k_means.fit(final)

KMeans(n_clusters=5, n_init=12)

In [82]:
k_means_labels = k_means.labels_
k_means_labels

array([3, 3, 3, 0, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 2, 0, 3, 2, 3, 3, 3, 3,
       4, 4, 3, 1, 1, 0, 2, 0, 3, 4, 2, 3, 3, 3, 3, 2, 2, 3, 2, 3, 2, 3,
       4, 2, 0, 3, 0, 4, 0, 4, 2, 2, 4, 0, 2, 1, 4, 1, 1, 2, 0, 0, 4, 2,
       2, 0, 0, 2, 2, 3, 2, 3, 2, 3, 0, 1, 3, 0, 2, 3, 2, 2, 1, 1, 0, 0,
       2, 2, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 2, 1])

In [83]:
labels = k_means_labels.tolist()
print(labels)

[3, 3, 3, 0, 3, 3, 3, 3, 3, 2, 2, 2, 2, 3, 2, 0, 3, 2, 3, 3, 3, 3, 4, 4, 3, 1, 1, 0, 2, 0, 3, 4, 2, 3, 3, 3, 3, 2, 2, 3, 2, 3, 2, 3, 4, 2, 0, 3, 0, 4, 0, 4, 2, 2, 4, 0, 2, 1, 4, 1, 1, 2, 0, 0, 4, 2, 2, 0, 0, 2, 2, 3, 2, 3, 2, 3, 0, 1, 3, 0, 2, 3, 2, 2, 1, 1, 0, 0, 2, 2, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 2, 1]


# Vizualization

In [84]:
label_df = DataFrame(labels, columns=['Labels'])
vizualization_df = ml_df.merge(label_df, left_index=True, right_index=True)
vizualization_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,South Indian Restaurant,Filipino Restaurant,Train Station,Greek Restaurant,Thai Restaurant,...,Shopping Plaza,State / Provincial Park,IT Services,Department Store,College Administrative Building,Newsstand,Playground,School,Beer Store,Labels
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,3
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,3
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,1,0,0,0,...,2,0,0,1,0,0,0,2,0,3
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,0,1,0,0,...,0,0,0,0,0,0,2,1,0,3


In [85]:
def assign_color(x):
    #function takes a number in range [0,4] and returns a string based on the number
    color = {0: "black", 1: "red", 2: "blue", 3: "green", 4:"purple"}
    if x >= 0 and x <= 4:
        return color[x]
    else:
        return "color out of range"

In [86]:
#toronto Coordinates for the map
toronto_lat = 43.651070
toronto_lon = -79.347015
toronto_map = folium.Map(location=[toronto_lat, toronto_lon], zoom_start=10)
neighbourhoods = folium.map.FeatureGroup()

#adding colors to each neighbourhood by its k-means cluster label. 
for lat, lon, lab in zip(vizualization_df["Latitude"], vizualization_df["Longitude"], vizualization_df["Labels"]):
    neighbourhoods.add_child(
        folium.features.CircleMarker(
                [lat, lon],
                radius=5, # define how big you want the circle markers to be
                color= assign_color(lab),
                fill=True,
                fill_color= assign_color(lab),
                fill_opacity=0.6
            )
        )
toronto_map.add_child(neighbourhoods)

In [61]:
assign_color(5)

'color out of range'